In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.transforms.functional as T
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils
# import matplotlib.pyplot as plt
# import keras
# from keras.layers import Input, merge
# from keras.layers import Convolution2D , concatenate ,Conv2D,Dense
# from keras.layers import Activation,Dropout, GlobalAveragePooling2D
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.utils import  to_categorical
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt
import torch.optim.lr_scheduler as tr

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

#from google.colab import drive                IF you are using COLAB

In [ ]:
# Use this if you are working on COLAB
# This will prompt for authorization.
#drive.mount('/content/drive')

In [2]:
def extract_data(x_data_filepath, y_data_filepath):
    X = np.load(x_data_filepath)
    y = np.load(y_data_filepath)
    return X, y

In [3]:
def data_visualization(images,labels):
    """
    Visualize 6 pictures per class using your prefered visulization library (matplotlib, etc)

    Args:
        images: training images in shape (num_images,3,image_H,image_W)
        labels: training labels in shape (num_images,)
    """
    label0=np.asarray(np.where(labels==0))
    label1=np.asarray(np.where(labels==1))
    label2=np.asarray(np.where(labels==2))
    label3=np.asarray(np.where(labels==3))
    label4=np.asarray(np.where(labels==4))
    

    train_img = images.transpose()
    for i in range(6):
        plt.figure()
        print(i)
        plt.imshow(train_img[:,:,:,label0[0][i]].swapaxes(-3,-2))
        plt.figure()
        plt.imshow(train_img[:,:,:,label1[0][i]].swapaxes(-3,-2))
        plt.figure()
        plt.imshow(train_img[:,:,:,label2[0][i]].swapaxes(-3,-2))
        plt.figure()
        plt.imshow(train_img[:,:,:,label3[0][i]].swapaxes(-3,-2))
        plt.figure()
        plt.imshow(train_img[:,:,:,label4[0][i]].swapaxes(-3,-2))
        plt.show()
#     pass

In [4]:
############################################################
# Extracting and loading data
############################################################
class Dataset(Dataset):
    def __init__(self, X, y):
        self.len = len(X)           
        if torch.cuda.is_available():
            self.x_data = torch.from_numpy(X).float().cuda()
            self.y_data = torch.from_numpy(y).long().cuda()
        else:
            self.x_data = torch.from_numpy(X).float()
            self.y_data = torch.from_numpy(y).long()
    
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

In [5]:
def create_validation(x_train,y_train):
    
    data = Dataset(x_train,y_train)
    num_train = len(x_train)
    indices = np.array((range(num_train)))
    split =  int(np.round(0.8* num_train))
    np.random.shuffle(indices)
    train_idx ,test_idx= indices[:split] , indices[split:]
    new_x_train = data.x_data[train_idx]
    new_y_train = data.y_data[train_idx]
    x_val = data.x_data[test_idx]
    y_val= data.y_data[test_idx]
      
    
    return new_x_train.numpy(),new_y_train.numpy(),x_val.numpy(),y_val.numpy()

In [ ]:
############################################################
# Feed Forward Neural Network
############################################################
class FeedForwardNN(nn.Module):
    """ 
        (1) Use self.fc1 as the variable name for your first fully connected layer
        (2) Use self.fc2 as the variable name for your second fully connected layer
    """
    def __init__(self):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(16320,2000)
        
        self.fc2 = nn.Linear(2000,5)
    def forward(self, x):
        # input size is flattened 
        x = x.view(x.size(0),-1)
        x_out = F.relu(self.fc1(x))
        # using this layer input as output for the other 
        out = self.fc2(x_out)
        

        return out

    """ 
        Please do not change the functions below. 
        They will be used to test the correctness of your implementation 
    """
    def get_fc1_params(self):
        return self.fc1.__repr__()
    
    def get_fc2_params(self):
        return self.fc2.__repr__()

In [ ]:
############################################################
# Convolutional Neural Network
############################################################
class ConvolutionalNN(nn.Module):
    """ 
        (1) Use self.conv1 as the variable name for your first convolutional layer
        (2) Use self.pool1 as the variable name for your first pooling layer
        (3) Use self.conv2 as the variable name for your second convolutional layer
        (4) Use self.pool2 as the variable name for you second pooling layer  
        (5) Use self.fc1 as the variable name for your first fully connected laye
        (6) Use self.fc2 as the variable name for your second fully connected layer
    """
    def __init__(self):
        super(ConvolutionalNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), stride=1, padding=0)
       
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3  = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3) , stride = 1, padding = 0)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(p=0.5)
        self.dropout1 = nn.Dropout(p=0.2)
        
        size1= (32*14*19)
        size2 = 3072
        # for autograder , typed itin 
        self.fc1 = nn.Linear(8512, 200)
        self.fc2 = nn.Linear(200,5)
    def forward(self, x):
        x_numpy_img = x.cpu().numpy()
        x= normalize_image(x_numpy_img)
        x.cuda()
        # conv1
#         out = self.conv1(x)
        out = F.relu(self.conv1(x))
        #maxpool                     
        out = self.pool1(out)
        #conv2 
#         out = self.conv2(out)
        out = F.relu(self.conv2(out))
        #maxpool2
        out = self.pool2(out)
        #conv3 
        out = F.relu(self.conv3(out))
        #maxpool3
        out = self.pool3(out)
        
        #out from conv is ()
        out = out.view(out.size(0), -1)
        #droputlayer
        out = self.dropout(out)
        # fully connected 1 
        out = self.fc1(out)
        out = F.relu(out)
        out = self.dropout1(out)
        #fully connected 2 
        out =self.fc2(out)
                               
        return out
      
    """ 
        Please do not change the functions below. 
        They will be used to test the correctness of your implementation
    """
    
    def get_conv1_params(self):
        return self.conv1.__repr__()
    
    def get_pool1_params(self):
        return self.pool1.__repr__()

    def get_conv2_params(self):
        return self.conv2.__repr__()
      
    def get_pool2_params(self):
        return self.pool2.__repr__()
      
    def get_fc1_params(self):
        return self.fc1.__repr__()
    
    def get_fc2_params(self):
        return self.fc2.__repr__()

In [ ]:
def normalize_image(image):
    """
    Normalize each input image

    Args:
    image: the input image in shape (3,image_H,image_W)
    Returns:
    norimg: the normalized image in the same shape as the input
    """
    
    
    img2 = image.copy()

    rmean , rvar=  np.mean(img2[0 ,:,:]), np.std(img2[0 ,:,:])
    gmean , gvar=  np.mean(img2[1,:,:]) , np.std(img2[1,:,:])
    bmean , bvar=  np.mean(img2[2,:,:]), np.std(img2[2,:,:])
#     image = image.cuda()
    
#     print(type(rmean))
#     print(torch.tensor(image).dtype)
    img2[0,:,:] = (img2[0,:,:] - rmean)/ rvar 
    img2[1,:,:] = (img2[1,:,:] - gmean)/ gvar 
    img2[2,:,:] = (img2[2,:,:] - bmean)/ bvar 
    
    normig = torch.tensor(img2)
#     normig= normig.type(torch.cuda.FloatTensor)
    
    return normig.numpy()

In [ ]:
############################################################
# Optimized Neural Network
############################################################
class OptimizedNN(nn.Module):
    def __init__(self):
        super(OptimizedNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=0)
       
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3  = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3 , stride = 1, padding = 0)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(p=0.5)
        self.dropout1 = nn.Dropout(p=0.2)
        
        size= (32*14*19)     
        self.fc1 = nn.Linear(3072, 200)
        self.fc2 = nn.Linear(200,5)                
        

    def forward(self, x):
        x_numpy_img = x.cpu().numpy()
        x= normalize_image(x_numpy_img)
        x.cuda()
        # conv1
#         out = self.conv1(x)
        out = F.relu(self.conv1(x))
        #maxpool                     
        out = self.maxpool1(out)
        #conv2 
#         out = self.conv2(out)
        out = F.relu(self.conv2(out))
        #maxpool2
        out = self.maxpool2(out)
        #conv3 
        out = F.relu(self.conv3(out))
        #maxpool3
        out = self.maxpool3(out)
        
        #out from conv is ()
        out = out.view(out.size(0), -1)
        #droputlayer
        out = self.dropout(out)
        # fully connected 1 
        out = self.fc1(out)
        out = F.relu(out)
        out = self.dropout1(out)
        #fully connected 2 
        out =self.fc2(out)        
        return out
      

In [ ]:
def train_val_NN(neural_network, train_loader, validation_loader, loss_function, optimizer,num_epochs):
    correct =0 
    correct_val = 0
    loss_np = np.zeros((num_epochs,1))
    train_loss = []
    valid_loss=[]
    accuracy = np.zeros((num_epochs+1,1))
    val_accuracy=np.zeros((num_epochs+1,1))
    i=0
    
    
    for epoch in range(0,num_epochs+1): 
        correct =0 
        correct_val = 0
        temp_acc =0
        
    ## training part 
        neural_network.train()
        for data, target in train_loader:
#             print(data.shape)
            optimizer.zero_grad()
            output = neural_network(data)
#             print(output.is_cuda)
            target =target.cuda()
            target = target.long()
            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            
        neural_network.eval()
        
        for data, target in test_loader:
            
            output = neural_network(data)
            loss = loss_function(output, target)
            valid_loss.append(loss.item())
#             print(output.is_cuda)
            pred = output.cpu()
            pred = pred.data.max(1, keepdim=True)[1]
#             print(pred.is_cuda)
            target1 = target.cpu()
            correct_val += pred.eq(target1.data.view_as(pred)).sum()
        print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))
    
        temp_acc =correct.item()/len(train_loader.dataset)
        accuracy[i]= temp_acc
        loss_np = valid_loss
        train_loss
        temp_val_acc =correct_val.item()/len(test_loader.dataset)
        val_accuracy[i]= temp_val_acc
        i+=1
    return accuracy,loss_np,val_accuracy, train_loss

In [ ]:
def test_NN(neural_network, test_loader):
  
    
    #taking testloader and getting each image 
    
    neural_network.eval()
    Preds = torch.LongTensor().cuda()
    for i, images in enumerate(test_loader):
        # each time one image is printed in testloader 
        data= images
#         print(data.shape)
        output = neural_network(data)
       # print(data.shape)
        pred = torch.max(output.data, 1)[1]
#         print(pred.is_cuda)
        Preds = torch.cat((Preds, pred), dim = 0 )
        
    
    return Preds

In [ ]:
# testing for test and making prediction file 

# test_images = np.load('data/images_test.npy')
# print(test_images.shape)
# test_loader = torch.utils.data.DataLoader(test_images , batch_size = 64, shuffle=False)
# Preds = test_NN(nnk, test_loader)

In [ ]:
# with open('HW4_preds.txt','w',encoding = 'utf8') as file:
#     for i in range(len(test_loader.dataset)):
#         file.write(str(Preds[i].item()))
#         file.write('\n')

In [ ]:
# Runnig Optimized Neural Network 

# run the optimized nueral network
# torch.cuda.empty_cache()
# images ,labels = extract_data( 'data/images_train.npy','data/labels_train.npy')

# new_x_train,new_y_train,x_val, y_val = create_validation(images,labels)

# print(type(new_x_train))
# print(type(new_y_train))

# train = torch.utils.data.TensorDataset(new_x_train,new_y_train)
# test = torch.utils.data.TensorDataset(x_val,y_val)

# train_loader = torch.utils.data.DataLoader(train, batch_size=64)
# test_loader = torch.utils.data.DataLoader(test, batch_size=64)


# neural_network = OptimizedNN()
# neural_network=neural_network.cuda()
# loss_function = torch.nn.CrossEntropyLoss().cuda()

# optimizer = torch.optim.Adam(neural_network.parameters(),lr =0.001)
# num_epochs = 40



# acc, los , val_acc = train_val_NN(neural_network, train_loader, test_loader, loss_function, optimizer,num_epochs)

# print(acc[-1])
# print(val_acc[-1])
# print(np.mean(acc))
# print(np.mean(val_acc))


In [ ]:
# for plotting the loss 
# fig = plt.figure()
# ax = plt.subplot(111)
# ax.plot(t, val_accs_np[:-1], marker='x')
# # ax.plot(t, loss_np, label='Validation-Accuracy with normalization')
# plt.title('Loss on Validation ')
# ax.legend()
# ax.set_xlabel('Epochs')
# plt.show()
# print(val_accs[-1])

